In [1]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report

# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler

from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.tokenize import word_tokenize
import gensim.downloader
import io
from keras_preprocessing.text import tokenizer_from_json
from gensim.models import Word2Vec


D:\anaconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set the seeder to have as stable random operations as possible

In [2]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
embedding_algorithms = ["w2v", "bert", "codebert"]
embedding_algorithm = embedding_algorithms[0]

In [4]:
representations = ["bow", "sequences"]
representation_form = representations[1]

In [5]:
# binary or multi-class
multi = True

In [6]:
train_w2v = False

Read data

In [7]:
if representation_form == "bow":
    data = pd.read_csv('bow_data.csv') # bow
else:
    data = pd.read_csv('sequences_data.csv') # sequences of tokens

In [8]:
data.head()

,Vulnerability,Category,Length
0,"f""str$id""""str$id""""str$id"" ...",sql_injection,9
1,client.listentcp() proxy = proxy(proxy_...,xsrf,8
2,"from django.http import httpresponse, httpresp...",open_redirect,9
3,"def write_preset(conn, queryin, descriptin):\t...",sql_injection,175
4,update_query = self.up...,sql_injection,14


In [9]:
if representation_form == "bow":
    bow_size = 237 # number of columns that stand as features

In [10]:
np.max(data["Length"])

392

In [11]:
if representation_form == "sequences":
    word_counts = data["Vulnerability"].apply(lambda x: len(x.split()))
    max_length = word_counts.max()
    print("Maximum number of words:", max_length)


Maximum number of words: 392


In [12]:
label_frequencies = data['Category'].value_counts()
print("Label Frequencies:\n", label_frequencies)
print("Total samples ", len(data))

Label Frequencies:
 sql_injection            1424
xsrf                      976
command_injection         721
path_disclosure           481
open_redirect             442
remote_code_execution     334
xss                       145
Name: Category, dtype: int64
Total samples  4523


In [13]:
if multi == True:
    n_categories = len(label_frequencies) # 7
    n_categories

Word Embedding

Word2Vec - load pre-trained word2vec embeddings - NL knowledge - static embeddings

In [14]:
if representation_form == "sequences":

    # Download the Punkt tokenizer models if not already downloaded
    nltk.download('punkt')

    tokenized_list = [word_tokenize(sentence) for sentence in data["Vulnerability"].tolist()]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iliaskaloup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BERT - load pre-trained bert embeddings - NL knowledge - contextual embeddings

In [15]:
if embedding_algorithm == "bert" and representation_form == "sequences": 
    model_variation = "bert-base-uncased" # "roberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_variation)
    bert = TFAutoModel.from_pretrained(model_variation)
    
    bert_embeddings = bert.get_input_embeddings()
    embedding_matrix = bert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

In [16]:
if embedding_algorithm == "bert" and representation_form == "sequences":
    sentences = data["Vulnerability"].tolist()
    sequences = [tokenizer.encode(sente, truncation=True, add_special_tokens=False, return_tensors="tf").numpy() for sente in sentences] # Tokenize the complete sentences
    
    lines_pad = []
    for seq in sequences:
        lines_pad.append(seq[0])
    
    lines_pad = pad_sequences(lines_pad, padding = 'post', maxlen = 512)

CodeBERT - load pre-trained codebert embeddings - PL knowledge - contextual embeddings

In [17]:
if embedding_algorithm == "codebert" and representation_form == "sequences": 
    model_variation = "microsoft/codebert-base"
    tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=True)
    codebert = TFAutoModel.from_pretrained(model_variation)
    
    codebert_embeddings = codebert.get_input_embeddings()
    embedding_matrix = codebert_embeddings.weights[0].numpy()
    num_words = len(embedding_matrix)
    print(num_words)
    dim = len(embedding_matrix[0])
    print(dim)

In [18]:
def padSequences(sequences, max_len):
    lines_pad = []
    for sequence in sequences:
        seq = sequence['input_ids'].numpy()[0]
        if len(seq) < max_len:
            for i in range(len(seq), max_len):
                seq = np.append(seq, 0)
        lines_pad.append(seq)
    return lines_pad


In [19]:
if embedding_algorithm == "codebert" and representation_form == "sequences": 

    sentences = data["Vulnerability"].tolist()
    sequences = [tokenizer(sente, return_tensors="tf", truncation=True, add_special_tokens=False) for sente in sentences]
    
    lines_pad = padSequences(sequences, 512)
    lines_pad = [arr.tolist() for arr in lines_pad]
    lines_pad = np.array(lines_pad)

RNN model, LSTM specifically

In [20]:
def buildMLP(n_inputs, n_outputs):
    nIn = n_inputs
    model = Sequential()
    #model.add(Dense(256, input_dim=nIn, kernel_constraint=max_norm(3), bias_constraint=max_norm(3)))#hidden
    model.add(Dense(256, input_dim=nIn))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_outputs)) #binary classification
    model.add(Activation('softmax'))#Output # sigmoid # softmax
    optimizer = optimizers.Adam(learning_rate=0.001) #, epsilon=1e-8, decay=1e-6
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer) # 'binary_crossentropy' # "sparse_categorical_crossentropy"
    return model

In [21]:
# Evaluation functions
def recall_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

def precision_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

def f1_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f1 = 2*((prec*rec)/(prec+rec+K.epsilon()))
    return f1

def f2_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f2 = 5*((prec*rec)/(4*prec+rec+K.epsilon()))
    return f2

In [22]:
def vectorize(words, w2v_vectors):
    words_vecs = [w2v_vectors[word] for word in words if word in w2v_vectors]
    if len(words_vecs) == 0:
        return np.zeros(300)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [23]:
w2v_vectors = gensim.downloader.load('word2vec-google-news-300')

Cross-Validation

Multi-class Classification: Categorization of all detected vulnerabilities

In [24]:
if multi == True:
    
    # Convert categories to numerical indexes
    category_numerical_indexes, unique_categories = data["Category"].factorize()

    # Create a dictionary mapping each category to its numerical index
    category_to_index = {category: index for index, category in enumerate(unique_categories)}

    # Update the categories in the DataFrame with their numerical indexes
    data["Category_Index"] = data["Category"].map(category_to_index)


Use DL models and sequences of tokens code representation

In [25]:
def runDLCrossVal_multi(X, y, w2v_vectors, max_len, seed):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
        
        X_train = np.array([vectorize(sentence, w2v_vectors) for sentence in X_train])
        X_test = np.array([vectorize(sentence, w2v_vectors) for sentence in X_test])
        
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        myModel = buildMLP(X_train.shape[1], n_categories)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [26]:
if multi == True and representation_form == "sequences" and train_w2v == False:
    runDLCrossVal_multi(np.array(tokenized_list), data["Category_Index"].values, w2v_vectors, 512, seed)

D:\anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Training...
fold number=  1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               77056     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)             

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               77056     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              


Epoch 00038: val_loss did not improve from 1.03266
Epoch 39/100
64/64 [==============================] - 0s 4ms/step - loss: 0.7033 - val_loss: 1.0783

Epoch 00039: val_loss did not improve from 1.03266
Epoch 40/100
64/64 [==============================] - 0s 4ms/step - loss: 0.7008 - val_loss: 1.0615

Epoch 00040: val_loss did not improve from 1.03266
Epoch 41/100
64/64 [==============================] - 0s 4ms/step - loss: 0.6874 - val_loss: 1.1116

Epoch 00041: val_loss did not improve from 1.03266
Epoch 42/100
64/64 [==============================] - 0s 4ms/step - loss: 0.6956 - val_loss: 1.0454

Epoch 00042: val_loss did not improve from 1.03266
Epoch 43/100
64/64 [==============================] - 0s 5ms/step - loss: 0.6821 - val_loss: 1.0456

Epoch 00043: val_loss did not improve from 1.03266
Epoch 44/100
64/64 [==============================] - 0s 5ms/step - loss: 0.6706 - val_loss: 1.0951

Epoch 00044: val_loss did not improve from 1.03266
Epoch 45/100
64/64 [================

64/64 [==============================] - 0s 5ms/step - loss: 0.7747 - val_loss: 1.1875

Epoch 00025: val_loss did not improve from 1.16131
Epoch 26/100
64/64 [==============================] - 0s 5ms/step - loss: 0.7589 - val_loss: 1.2242

Epoch 00026: val_loss did not improve from 1.16131
Epoch 00026: early stopping
[[100  21   4   0   4  12   2]
 [  3  84   1   0   5   4   0]
 [  7  17  12   1   4   3   0]
 [  3   4   1   4   2   1   0]
 [  3   9   0   0  19   1   1]
 [  7  10   4   0   1  47   3]
 [ 10  11   3   1   3   6  15]]
Accuracy:62.03%
Precision:62.03%
Recall:62.03%
F1 score:62.03%
              precision    recall  f1-score   support

           0       0.75      0.70      0.72       143
           1       0.54      0.87      0.66        97
           2       0.48      0.27      0.35        44
           3       0.67      0.27      0.38        15
           4       0.50      0.58      0.54        33
           5       0.64      0.65      0.64        72
           6       0.

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 256)               77056     
_________________________________________________________________
activation_16 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               32896     
_________________________________________________________________
activation_17 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)              

64/64 [==============================] - 1s 6ms/step - loss: 1.7050 - val_loss: 1.5626

Epoch 00001: val_loss improved from inf to 1.56260, saving model to best_model.h5
Epoch 2/100
64/64 [==============================] - 0s 5ms/step - loss: 1.5153 - val_loss: 1.4673

Epoch 00002: val_loss improved from 1.56260 to 1.46726, saving model to best_model.h5
Epoch 3/100
64/64 [==============================] - 0s 5ms/step - loss: 1.4010 - val_loss: 1.3776

Epoch 00003: val_loss improved from 1.46726 to 1.37764, saving model to best_model.h5
Epoch 4/100
64/64 [==============================] - 0s 5ms/step - loss: 1.3099 - val_loss: 1.3303

Epoch 00004: val_loss improved from 1.37764 to 1.33032, saving model to best_model.h5
Epoch 5/100
64/64 [==============================] - 0s 5ms/step - loss: 1.2565 - val_loss: 1.2962

Epoch 00005: val_loss improved from 1.33032 to 1.29621, saving model to best_model.h5
Epoch 6/100
64/64 [==============================] - 0s 5ms/step - loss: 1.1985 - val_

64/64 [==============================] - 0s 4ms/step - loss: 1.1456 - val_loss: 1.2991

Epoch 00007: val_loss improved from 1.31946 to 1.29908, saving model to best_model.h5
Epoch 8/100
64/64 [==============================] - 0s 5ms/step - loss: 1.1134 - val_loss: 1.2778

Epoch 00008: val_loss improved from 1.29908 to 1.27778, saving model to best_model.h5
Epoch 9/100
64/64 [==============================] - 0s 5ms/step - loss: 1.0756 - val_loss: 1.2669

Epoch 00009: val_loss improved from 1.27778 to 1.26692, saving model to best_model.h5
Epoch 10/100
64/64 [==============================] - 0s 5ms/step - loss: 1.0449 - val_loss: 1.2549

Epoch 00010: val_loss improved from 1.26692 to 1.25485, saving model to best_model.h5
Epoch 11/100
64/64 [==============================] - 0s 4ms/step - loss: 1.0136 - val_loss: 1.2679

Epoch 00011: val_loss did not improve from 1.25485
Epoch 12/100
64/64 [==============================] - 0s 4ms/step - loss: 0.9870 - val_loss: 1.2632

Epoch 00012: v


Epoch 00011: val_loss did not improve from 1.17567
Epoch 12/100
64/64 [==============================] - 0s 5ms/step - loss: 0.9953 - val_loss: 1.1572

Epoch 00012: val_loss improved from 1.17567 to 1.15719, saving model to best_model.h5
Epoch 13/100
64/64 [==============================] - 0s 5ms/step - loss: 0.9558 - val_loss: 1.1510

Epoch 00013: val_loss improved from 1.15719 to 1.15103, saving model to best_model.h5
Epoch 14/100
64/64 [==============================] - 0s 5ms/step - loss: 0.9429 - val_loss: 1.1701

Epoch 00014: val_loss did not improve from 1.15103
Epoch 15/100
64/64 [==============================] - 0s 5ms/step - loss: 0.9154 - val_loss: 1.1976

Epoch 00015: val_loss did not improve from 1.15103
Epoch 16/100
64/64 [==============================] - 0s 5ms/step - loss: 0.9065 - val_loss: 1.1697

Epoch 00016: val_loss did not improve from 1.15103
Epoch 17/100
64/64 [==============================] - 0s 5ms/step - loss: 0.8817 - val_loss: 1.1562

Epoch 00017: val_

64/64 [==============================] - 0s 5ms/step - loss: 0.8391 - val_loss: 1.0313

Epoch 00021: val_loss did not improve from 0.98810
Epoch 22/100
64/64 [==============================] - 0s 5ms/step - loss: 0.8190 - val_loss: 1.0330

Epoch 00022: val_loss did not improve from 0.98810
Epoch 23/100
64/64 [==============================] - 0s 5ms/step - loss: 0.8101 - val_loss: 1.0187

Epoch 00023: val_loss did not improve from 0.98810
Epoch 24/100
64/64 [==============================] - 0s 5ms/step - loss: 0.7968 - val_loss: 0.9970

Epoch 00024: val_loss did not improve from 0.98810
Epoch 25/100
64/64 [==============================] - 0s 5ms/step - loss: 0.7923 - val_loss: 1.0018

Epoch 00025: val_loss did not improve from 0.98810
Epoch 26/100
64/64 [==============================] - 0s 5ms/step - loss: 0.7735 - val_loss: 0.9833

Epoch 00026: val_loss improved from 0.98810 to 0.98333, saving model to best_model.h5
Epoch 27/100
64/64 [==============================] - 0s 5ms/step 

One more option for training your own word2vec vectors

In [27]:
def trainWord2Vec(tokenized_list, max_length, dim):
    # this should be executed only in the training set during cross-validation
    w2v_model = Word2Vec(sentences=tokenized_list, vector_size=dim, window=5, min_count=1, workers=4, epochs=4)
    #w2v_model.save("python_word2vec.model")
    #w2v_model = Word2Vec.load("python_word2vec.model")

    #     fileEmb = 'w2v_embeddingsIters.txt'
    #     w2v_vectors.save_word2vec_format(fileEmb, binary=False)

    #     embeddings_index = {}
    #     f = open(os.path.join('', fileEmb), encoding="utf-8")
    #     for line in f:    
    #         values = line.split()
    #         word = values[0]
    #         coefs = np.asarray(values[1:])
    #         embeddings_index[word] = coefs   
    #     f.close()


    tokenizer_obj = Tokenizer()   
    tokenizer_obj.fit_on_texts(tokenized_list)

    tokenizer_json = tokenizer_obj.to_json()
    tokenizerFile = 'w2v_new_tokenizer.json'
    with io.open(tokenizerFile, 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))

    with open(tokenizerFile) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)

    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    word_index = tokenizer_obj.word_index

    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)


    num_words = len(word_index) + 1 # +1 for the unknown-zeros
    embedding_matrix = np.zeros((num_words, dim))
    for word, i in word_index.items():
        if i > num_words:
            continue
        #embedding_vector = embeddings_index.get(word)
        embedding_vector = w2v_model.wv[word] if word in w2v_model.wv else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return lines_pad, embedding_matrix, num_words, tokenizerFile


In [28]:
def testWord2Vec(tokenized_list, tokenizer_path, max_length):
    
    with open(tokenizer_path) as f:
        dataTokenizer = json.load(f)
        tokenizer_obj = tokenizer_from_json(dataTokenizer)
        
    sequences = tokenizer_obj.texts_to_sequences(tokenized_list)
    
    lines_pad = pad_sequences(sequences, padding = 'post', maxlen = max_length)
    
    return lines_pad

In [29]:
def runDLCrossValW2v_multi(X, y, max_len, dim, seed):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
        
        X_train, embedding_matrix, num_words, tokenizerFile = trainWord2Vec(X_train, max_len, dim)
        
        X_test = testWord2Vec(X_test, tokenizerFile, max_len)
        
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, n_categories)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predScores = myModel.predict(X_test)
        predictions = np.argmax(predScores, axis=1)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [30]:
if train_w2v == True and multi == True and representation_form == "sequences":
    dim = 300
    runDLCrossValW2v_multi(np.array(tokenized_list), data["Category_Index"].values, 512, dim, seed)